In [ ]:
import midii
import copy
from pathlib import Path

In [ ]:
mid = midii.MidiFile(
    midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
)
Path(mid.filename).name

'ba_05688_-4_a_s02_m_02.mid'

In [5]:
mid_q32 = copy.deepcopy(mid)
mid_q32.quantize(unit="32")

In [6]:
def mean_grid_alignment_error(items, unit_beat_ticks):
    total_error = 0
    for x in items:
        error = x['end'] % unit_beat_ticks
        if error < unit_beat_ticks/2:
            total_error += error
        else:
            total_error += unit_beat_ticks - error
        error = x['start'] % unit_beat_ticks
        if error < unit_beat_ticks/2:
            total_error += error
        else:
            total_error += unit_beat_ticks - error
    return total_error/len(items)

In [7]:

print(f"Mean Grid Alignment Error={mean_grid_alignment_error(mid.to_json(), 15):.3f}")
print(f"Mean Grid Alignment Error={mean_grid_alignment_error(mid_q32.to_json(), 15)}")

Mean Grid Alignment Error=7.676
Mean Grid Alignment Error=0.0
